In this project I will be analyzing data from used cars taken from eBay Kleinanzeigen, a German eBay website. A dictionary for the data is shown below:

- `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing
- `price` - The price on the ad to sell the car.
-`abtest` - Whether the listing is included in an A/B test.
-`vehicleType` - The vehicle Type.
-`yearOfRegistration` - The year in which the car was first registered.
-`gearbox` - The transmission type.
-`powerPS` - The power of the car in PS.
-`model` - The car model name.
-`kilometer` - How many kilometers the car has driven.
-`monthOfRegistration` - The month in which the car was first registered.
-`fuelType` - What type of fuel the car uses.
-`brand` - The brand of the car.
-`notRepairedDamage` - If the car has a damage which is not yet repaired.
-`dateCreated` - The date on which the eBay listing was created.
-`nrOfPictures` - The number of pictures in the ad.
-`postalCode` - The postal code for the location of the vehicle.
-`lastSeenOnline` - When the crawler saw this ad last online.

In [1]:
#import numpy with alias np 
#import pandas with alias pd

import numpy as np
import pandas as pd

#read autos.csv into Jupyter as a dataframe using pandas

autos = pd.read_csv(r'C:\Users\david\Desktop\autos.csv', encoding='latin1')

In [2]:
#use the .info() method to return basic info of each column
#use the .head() method to return the first five rows of our dataframe

autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Before anything I'm going to rename the columns into something more sensible as well as make the column names snakecase (lowercase).

In [3]:
#renames each columnn from first to last
#returns column names

autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code', 'last_seen']
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

Now that I've changed the column names into something that's easier to read lets dig into the information shown above in our second cell. Immediately I can see two things:

- Our dataset contains 20 columns and 50,000 rows with a couple thousand missing values, nothing that should majorly impair my analysis
- The columns ```price``` and ```odometer``` are represented as dtype objects, probably due to the presence of ```$``` and ```km```

Lets remove those substrings and convert those two columns into integer objects now.

---

## Exploring seller, offer_type and num_photos columns

In [4]:
#using method chaining we replace (or remove) '$' with '' (or nothing) as well as the commas using the .str.replace() method.
#we also convert the object type into integers with the .astype(int) method
#finally we print the first five rows to verify our code worked

autos['price'] = (autos['price']
                          .str.replace('$','')
                          .str.replace(',','')
                          .astype(int)
                          )

autos['price'].head()

C:\Users\david\AppData\Local\Temp\ipykernel_1468\2969186809.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  autos['price'] = (autos['price']


0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int32

In [5]:
#using method chaining we replace (or remove) 'km' with '' (or nothing) as well as the commas using the .str.replace() method.
#we also convert the object type into integers with the .astype(int) method
#finally we print the first five rows to verify our code worked

autos['odometer_km'] = (autos['odometer_km']
                         .str.replace('km','')
                         .str.replace(',','')
                         .astype(int)
                    )

autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int32

---

Now that we've cleaned those columns lets take a closer look at the other columns for any redundencies or errors that might be present

In [6]:
#passes the dataframe through the .describe() with the argument ```include='all'``` to return descriptive statistics 
#for all columns of the dataframe

autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


From the above table I can see three red flags:

- Column ```seller``` has 2 unique values with 49,999 frequency, meaning out of two of those values one repeats 49,999 times
- Column ```offer_type``` has 2 unique values with 49,999 frequency, meaning out of two of those values one repeats 49,999 times
- Column ```num_photos``` has no data, this needs to be further explored

My next steps are to:

1. Verify columns ```seller``` and ```offer_type``` have data redundancy 
2. Investigate column ```num_photos``` further and decide what to do

In [7]:
#prints value_counts for both the seller and offer_type columns

print(autos['seller'].value_counts(), '\n', '\n', autos['offer_type'].value_counts())

privat        49999
gewerblich        1
Name: seller, dtype: int64 
 
 Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


In [8]:
#find the sum of all unique values

autos['num_photos'].unique().sum()

0

In the above two cells I have verified that ```seller``` column has 49,999 private entries and a single gewerblich (German for "commercial) entry. The ```offer_type``` column also displays a similar case. Furthermore, I've found that the the column ```num_photos``` 0 entered for every row. 

All of these columns will be dropped for redundancy and null values.

In [9]:
#drops or removes specificed columns

autos = autos.drop(['num_photos', 'seller', 'offer_type'], axis=1)

## Exploring Odometer and Price Columns

Now that those columns are removed I will move onto exploring more of the data. Specifically, I will be looking for outliers in the ```odometer_km``` and ```price``` columns.

In [30]:
print('Number of unique values in odometer_km:', '\n', autos['odometer_km'].unique())
print('\n')
print('Frequency of unique values:', '\n', autos['odometer_km'].value_counts())

Number of unique values in odometer_km: 
 [150000  70000  50000  80000  10000  30000 125000  90000  20000  60000
   5000  40000 100000]


Frequency of unique values: 
 150000    30085
125000     4857
100000     2058
90000      1673
80000      1375
70000      1187
60000      1128
50000       993
40000       797
5000        785
30000       760
20000       742
10000       241
Name: odometer_km, dtype: int64


In [35]:
print('Number of unique values in price:', autos['price'].unique().shape)
print('\n')
print('Frequency of unique values in price:', '\n', autos['price'].value_counts(ascending=False))
print('\n')
print('Statistics for price:', autos['price'].describe())

Number of unique values in price: (2323,)


Frequency of unique values in price: 
 500      757
1500     696
2500     614
1200     606
1000     602
        ... 
4895       1
1960       1
20299      1
38760      1
16995      1
Name: price, Length: 2323, dtype: int64


Statistics for price: count     46681.000000
mean       5977.716801
std        9177.909479
min           1.000000
25%        1250.000000
50%        3100.000000
75%        7500.000000
max      350000.000000
Name: price, dtype: float64


In the above two cells we can see that the ```odometer_km``` column has numbers in the range you'd expect. However, the ```price``` seems to look good at first glance until we see that the maximum price listed for a vehicle is 100 million dollars. This is strange so lets explore this data a bit more.

In [12]:
#list the first 25 values of the price column in a descending order from greatest to least and list their frequency

autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

We can see that there are a couple of values that seem rather fishy, starting from one and sequestially counting up to nine or just maxing out the top price of the website. Clearly these are mistakes or fake posts, lets remove the values from that aren't in the range of 1 to 351,000.

In [13]:
#discard any values not inbetween the range of 1 and 351,00 for prices column.

autos = autos[autos['price'].between(1, 351000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

Now our statistical overview looks how we would expect it for used car sales. Next lets look at the date columns quantitatively and convert them into numerical data types if need be.

---

## Exploring date columns

In [14]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48565 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        48565 non-null  object
 1   name                48565 non-null  object
 2   price               48565 non-null  int32 
 3   ab_test             48565 non-null  object
 4   vehicle_type        43979 non-null  object
 5   registration_year   48565 non-null  int64 
 6   gearbox             46222 non-null  object
 7   power_ps            48565 non-null  int64 
 8   model               46107 non-null  object
 9   odometer_km         48565 non-null  int32 
 10  registration_month  48565 non-null  int64 
 11  fuel_type           44535 non-null  object
 12  brand               48565 non-null  object
 13  unrepaired_damage   39464 non-null  object
 14  ad_created          48565 non-null  object
 15  postal_code         48565 non-null  int64 
 16  last_seen           48

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


As shown in the output cells above we can see that there are five date columns with ```registration_month``` and ```registration_year``` being numerical data types.


List of date columns:
- ```date_crawled```
-```registration_year```
-```registration_month```
-```ad_created```
-```last_seen```

Before we analyze ```registration_year``` and ```registration_month``` lets anaylze the string dates.

In [15]:
#return the first five rows of the columns specificed

autos[['date_crawled', 'ad_created', 'last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [16]:
#the goal of this code is to find the chronologically first date that the spider crawled and the last date it crawled

#selects the first 10 substrings of the date_crawled column
#returns a series of unique values and their relative frequencies 
#sorts a series by it's label in ascending manner

print(autos['date_crawled']
      .str[:10]
      .value_counts(normalize=True, dropna=False)
      .sort_index())

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64


In [17]:
#the goal of this code is to sort by relative frequency so we can determine if the rate at which the spider crawled per day
#is uniform or not

#selects the first 10 substrings of the date_crawled column
#returns a series of unique values and their relative frequencies 
#sorts a series by it's label in descending manner

print(autos['date_crawled']
      .str[:10]
      .value_counts(normalize=True, dropna=False)
      .sort_values()
     )

2016-04-07    0.001400
2016-04-06    0.003171
2016-03-18    0.012911
2016-04-05    0.013096
2016-03-06    0.014043
2016-03-13    0.015670
2016-03-05    0.025327
2016-03-24    0.029342
2016-03-16    0.029610
2016-03-27    0.031092
2016-03-25    0.031607
2016-03-17    0.031628
2016-03-31    0.031834
2016-03-10    0.032184
2016-03-26    0.032204
2016-03-23    0.032225
2016-03-11    0.032575
2016-03-22    0.032987
2016-03-09    0.033090
2016-03-08    0.033296
2016-04-01    0.033687
2016-03-30    0.033687
2016-03-29    0.034099
2016-03-15    0.034284
2016-03-19    0.034778
2016-03-28    0.034860
2016-04-02    0.035478
2016-03-07    0.036014
2016-04-04    0.036487
2016-03-14    0.036549
2016-03-12    0.036920
2016-03-21    0.037373
2016-03-20    0.037887
2016-04-03    0.038608
Name: date_crawled, dtype: float64


The above two cells indicate that this site was crawled for about a month, starting in March of 2016 and ending in April of 2016. The number of listings the spider crawled per day for a month remained approximately uniform.

In [18]:
#prints the first 10 substrings of unique values and their relative frequencies and sorts them by the index (also
#includes null values)

print(autos['last_seen']
      .str[:10]
      .value_counts(normalize=True, dropna=False)
      .sort_index()
     )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


The above cell shows us the the date the crawler recorded seeing any listing, which presumably means the car was sold as the listing was removed. However, looking towards the last few dates the crawler was crawling this set we can see a spike in the relative frequency of last seen recordings. 

This is most likely due to our crawling period ending, and cannot be indicitive of car sales.

In [19]:
print('Number of rows for ad_created:', autos['ad_created'].str[:10].unique().shape)

print(autos['ad_created']
      .str[:10]
      .value_counts(normalize=True, dropna=False)
      .sort_index())

Number of rows for ad_created: (76,)
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64


As shown above there are some ads which are as old as 9 months compared to the newest ads.

---

Lets explore ```registration_year``` and ```registration_month``` now that we've analyzed the string dates.

In [20]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Immediately we can see the minimum number for ```registration_year``` is 1000 and the maximum is 9999. This is not possible as these numbers represent the year. To combat this we'll remove any listings that are not in the range of 1900 to 2016 (as 1900 would be one of the first cars and 2016 is the year our data cuts off.) 

# review  ^^^

In [21]:
# comment

autos = autos[autos['registration_year'].between(1900, 2016)]

autos['registration_year'].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

Now that we've cleaned the data abit more we can see that the year in which the most listings created were in 2002.

---

## Exploring Price by Brand

Next I would like to see if I can find any correlation between the brand of car sold and the price it goes for. Let's explore the brand column.

In [22]:
autos['brand'].value_counts(normalize=True).sort_values(ascending=False)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

In the output cell above we can see the number of times each value has been founded in the ```brand``` column. With Volkswagens being the most common. For our purposes we'll take the brands representing more than 5% of the total listings.

In [23]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [24]:
brand_mean_price = {}

for brand in common_brands:
    brand_counts = autos[autos['brand'] == brand]
    mean_price = brand_counts['price'].mean()
    brand_mean_price[brand] = int(mean_price)
    
brand_mean_price

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2975,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749}

Of our top brands we can see that Audis, BMWs and Mercedes Benz are the most expensive. Volkswagens are middle of the road and Fords and Opels are the cheapest option.

---

Next I would like to analyze the average price of each of these brands in comparison with their average mileage to see if there is any correlation.

In [25]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_counts = autos[autos['brand'] == brand]
    mean_mileage = brand_counts['odometer_km'].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
brand_mean_mileage

{'volkswagen': 128707,
 'bmw': 132572,
 'opel': 129310,
 'mercedes_benz': 130788,
 'audi': 129157,
 'ford': 124266}

In [26]:
avg_brand_price = pd.Series(brand_mean_price).sort_values(ascending=False)
pd.DataFrame(avg_brand_price, columns=['mean_price'])

,mean_price
audi,9336
mercedes_benz,8628
bmw,8332
volkswagen,5402
ford,3749
opel,2975


In [27]:
avg_brand_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
pd.DataFrame(avg_brand_mileage, columns=['mean_mileage'])

,mean_mileage
bmw,132572
mercedes_benz,130788
opel,129310
audi,129157
volkswagen,128707
ford,124266


In [28]:
brand_info = pd.DataFrame(avg_brand_mileage,columns=['mean_mileage'])
brand_info['mean_price'] = avg_brand_price
brand_info

,mean_mileage,mean_price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


Overall the mileage of the vehicle doesn't seem to be a massive factor in the price as shown by the table above. It does seem like there is a small correlation with the more expensive the vehicle the more mileage, and the less expensive vehicles having lower mileage.